## Week 3 assignment Part 3

In [12]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
from collections import deque

In [13]:
LIMIT = 100
RADIUS = 500
NO_OF_CLUSTERS = 5
CLIENT_ID = 'Enter your ID'
CLIENT_SECRET = 'Enter secret ID'
VERSION = '20190528'

In [14]:
dataset = pd.read_csv('df_cord.csv')
dataset.head(5)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1C,43.784535,-79.160497
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1E,43.763573,-79.188711
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1G,43.770992,-79.216917
3,M1G,Scarborough,Woburn,M1H,43.773136,-79.239476
4,M1H,Scarborough,Cedarbrae,M1J,43.744734,-79.239476


In [15]:
dataset= dataset.dropna()

### Explore data collected with latitudes and longitudes

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
selected_dataset = dataset[dataset['Borough'].str.contains('Toronto')].reset_index(drop = True)

### Plot Data on Map using Folium

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = dataset

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [25]:
print("The neighborhood I'll explore is: {}".format(selected_dataset.loc[0, 'Neighborhood']))

The neighborhood I'll explore is: The Beaches


In [26]:
neighborhood_name = selected_dataset.loc[0, 'Neighborhood']
neighborhood_latitude = selected_dataset.loc[0, 'Latitude']
neighborhood_longitude = selected_dataset.loc[0, 'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.7116948, -79.41693559999999.


## explore 100 venues loacted in Beaches area

In [27]:
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
       '&client_secret={}&v={}&ll={},{}&radius={}&limit={}').format(CLIENT_ID, 
                                                                  CLIENT_SECRET, 
                                                                  VERSION, 
                                                                  neighborhood_latitude, 
                                                                  neighborhood_longitude, 
                                                                  RADIUS,
                                                                  LIMIT)
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5e948f2e006dce001b4eed87'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Lawrence Park South',
  'headerFullLocation': 'Lawrence Park South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7161948045, 'lng': -79.41072165393975},
   'sw': {'lat': 43.707194795499994, 'lng': -79.42314954606023}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5ae8cfe98496ca002c65d83a',
       'name': 'Ceiling Champions',
       'location': {'address': '505 The West Mall Suite #207',
        'lat': 43.71389073142895,
        'lng': -79.42070245742796,
        

###### Extract the information out of the result we got from the Foursquare API. I'll use the method get_category_type to get the category name.

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = result['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# Filter the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# Clean all column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ceiling Champions,Home Service,43.713891,-79.420702
1,Rosalind's Garden Oasis,Garden,43.712189,-79.411978


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


2 venues were returned by Foursquare.


### Names of venues in Neighborhood

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
list_of_venues = getNearbyVenues(selected_dataset['Neighborhood'],
                                 selected_dataset['Latitude'],
                                 selected_dataset['Longitude'],
                                 RADIUS)

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city


In [33]:
list_of_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,14,14,14,14,14,14
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,7,7,7,7,7,7
Central Bay Street,40,40,40,40,40,40
Church and Wellesley,23,23,23,23,23,23
Commerce Court / Victoria Hotel,12,12,12,12,12,12
Davisville,4,4,4,4,4,4
Davisville North,16,16,16,16,16,16
First Canadian Place / Underground city,3,3,3,3,3,3
Forest Hill North & West,2,2,2,2,2,2


In [34]:
print('There are {} uniques categories.'.format(len(list_of_venues['Venue Category'].unique())))

There are 183 uniques categories.


In [42]:
Toronto_OHE = pd.get_dummies(list_of_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = list_of_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Record Shop,Recording Studio,Restaurant,River,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Smoke Shop,Soup Place,Spa,Speakeasy,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [43]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Record Shop,Recording Studio,Restaurant,River,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Smoke Shop,Soup Place,Spa,Speakeasy,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.071429,0.071429,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.071429,0.000,0.0,0.071429,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.071429,0.0,0.0,0.0,0.071429,0.000,0.0,0.0,0.071429,0.0,0.0,0.071429,0.000,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.142857,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.071429,0.0,0.0,0.000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.071429,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000
1,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.000,0

In [44]:
Toronto_grouped.shape

(28, 184)

### Top 5 veneues in Neighborhood

In [59]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Cluster Labels  1.00
1       Gift Shop  0.14
2       Bookstore  0.07
3    Dessert Shop  0.07
4            Bank  0.07


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
               venue  freq
0     Cluster Labels  1.00
1         Beer Store  0.14
2       Liquor Store  0.14
3     Cosmetics Shop  0.14
4  Convenience Store  0.14


----Central Bay Street----
              venue  freq
0    Cluster Labels  1.00
1       Coffee Shop  0.08
2              Café  0.08
3       Pizza Place  0.05
4  Sushi Restaurant  0.05


----Church and Wellesley----
            venue  freq
0  Cluster Labels  1.00
1            Café  0.13
2  Breakfast Spot  0.09
3     Coffee Shop  0.09
4             Bar  0.04


----Commerce Court / Victoria Hotel----
                  venue  freq
0        Cluster Labels  1.00
1                  Café  0.17
2           Pizza Place  0.08
3  Fast Food Restaurant  0.0

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Neighborhood clustering

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cluster Labels,Gift Shop,Bank,Movie Theater,Cuban Restaurant,Dessert Shop,Breakfast Spot,Restaurant,Dog Run,Bookstore
1,CN Tower / King and Spadina / Railway Lands / ...,Cluster Labels,Beer Store,Convenience Store,Pizza Place,Cosmetics Shop,Café,Liquor Store,Coffee Shop,Fountain,Food Truck
2,Central Bay Street,Cluster Labels,Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Sushi Restaurant,Sandwich Place,Bookstore,Boutique
3,Church and Wellesley,Cluster Labels,Café,Breakfast Spot,Coffee Shop,Nightclub,Restaurant,Burrito Place,Pet Store,Performing Arts Venue,Climbing Gym
4,Commerce Court / Victoria Hotel,Cluster Labels,Café,Gym,Restaurant,Liquor Store,Bakery,Fast Food Restaurant,Pharmacy,Pet Store,Pizza Place


### Clustering

In [63]:
toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# Run K-Means clustering
kMeans = KMeans(n_clusters = NO_OF_CLUSTERS, random_state = 0).fit(Toronto_grouped_clustering)

In [64]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kMeans.labels_)

# Combine the data
toronto_merged = selected_dataset.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,M5N,43.711695,-79.416936,0.0,Home Service,Garden,Cluster Labels,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
1,M4K,East Toronto,The Danforth West / Riverdale,M5T,43.653206,-79.400049,1.0,Cluster Labels,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Farmers Market,Dumpling Restaurant
2,M4L,East Toronto,India Bazaar / The Beaches West,M5V,43.628947,-79.394420,1.0,Cluster Labels,Airport Terminal,Airport Lounge,Airport Service,Boutique,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court
3,M4M,East Toronto,Studio District,M5W,43.646435,-79.374846,1.0,Cluster Labels,Coffee Shop,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Hotel,Japanese Restaurant,Beer Bar,Cocktail Bar
4,M4N,Central Toronto,Lawrence Park,M5X,43.648429,-79.382280,1.0,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Asian Restaurant,Seafood Restaurant,Bar


In [71]:
toronto_merged.dropna()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,M5N,43.711695,-79.416936,0.0,Home Service,Garden,Cluster Labels,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
1,M4K,East Toronto,The Danforth West / Riverdale,M5T,43.653206,-79.400049,1.0,Cluster Labels,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Farmers Market,Dumpling Restaurant
2,M4L,East Toronto,India Bazaar / The Beaches West,M5V,43.628947,-79.394420,1.0,Cluster Labels,Airport Terminal,Airport Lounge,Airport Service,Boutique,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court
3,M4M,East Toronto,Studio District,M5W,43.646435,-79.374846,1.0,Cluster Labels,Coffee Shop,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Hotel,Japanese Restaurant,Beer Bar,Cocktail Bar
4,M4N,Central Toronto,Lawrence Park,M5X,43.648429,-79.382280,1.0,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Asian Restaurant,Seafood Restaurant,Bar
5,M4P,Central Toronto,Davisville North,M6A,43.718518,-79.464763,1.0,Cluster Labels,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Accessories Store
6,M4R,Central Toronto,North Toronto West,M6B,43.709577,-79.445073,2.0,Cluster Labels,Bakery,Pizza Place,Pub,Japanese Restaurant,Cupcake Shop,Dog Run,Distribution Center,Discount Store,Diner
7,M4S,Central Toronto,Davisville,M6C,43.693781,-79.428191,1.0,Cluster Labels,Hockey Arena,Trail,Playground,Field,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
8,M4T,Central Toronto,Moore Park / Summerhill East,M6E,43.689026,-79.453512,3.0,Cluster Labels,Park,Pool,Women's Store,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,M6G,43.669542,-79.422564,1.0,Cluster Labels,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Coffee Shop,Nightclub,Candy Store


##### Analyze clusters

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Home Service,Garden,Cluster Labels,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cluster Labels,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Farmers Market,Dumpling Restaurant
2,Cluster Labels,Airport Terminal,Airport Lounge,Airport Service,Boutique,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate,Airport Food Court
3,Cluster Labels,Coffee Shop,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Hotel,Japanese Restaurant,Beer Bar,Cocktail Bar
4,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gastropub,Asian Restaurant,Seafood Restaurant,Bar
5,Cluster Labels,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Accessories Store


In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Cluster Labels,Bakery,Pizza Place,Pub,Japanese Restaurant,Cupcake Shop,Dog Run,Distribution Center,Discount Store,Diner
28,Cluster Labels,Empanada Restaurant,Pizza Place,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Cluster Labels,Park,Pool,Women's Store,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Cluster Labels,Baseball Field,Locksmith,Deli / Bodega,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
